In [1]:
%pip install gradio pillow matplotlib requests google-generativeai



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U ipywidgets jupyter


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -U google-generativeai



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import json
import os
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt


# Keys / endpoints (user asked to keep the same placeholder keys)
AZURE_VISION_KEY = ""
AZURE_VISION_ENDPOINT = ""

# Translator placeholders (keeps style of placeholders you used)
AZURE_TRANSLATOR_KEY = ""
AZURE_TRANSLATOR_ENDPOINT = ""

# Gemini / Google GenAI placeholders (keeps the api_key string you used in the cells)
GENAI_API_KEY = ""

In [5]:
def analyze_image_azure(image_url=None, image_bytes=None):
    """Analyze image with Azure Computer Vision (v3.2 analyze). Returns parsed json.
    Accepts either an image_url or bytes (uploaded file).
    """
    analyze_url = f"{AZURE_VISION_ENDPOINT}/vision/v3.2/analyze"
    headers = {"Ocp-Apim-Subscription-Key": AZURE_VISION_KEY}
    params = {"visualFeatures":"ImageType,Description,Faces,Objects,Tags"}

    if image_url:
        payload = {"url": image_url}
        resp = requests.post(analyze_url, headers=headers, params=params, json=payload)
    elif image_bytes:
        # send binary image
        headers_bin = {"Ocp-Apim-Subscription-Key": AZURE_VISION_KEY, "Content-Type": "application/octet-stream"}
        resp = requests.post(analyze_url, headers=headers_bin, params=params, data=image_bytes)
    else:
        raise ValueError("Either image_url or image_bytes must be provided")

    resp.raise_for_status()
    return resp.json()

In [6]:
def plot_image_with_objects(image_pil, analysis_json):
    fig, ax = plt.subplots(figsize=(8,6))
    ax.imshow(image_pil)
    ax.axis('off')

    if 'objects' in analysis_json:
        for obj in analysis_json['objects']:
            rect = obj['rectangle']
            label = obj.get('object', '')
            patch = plt.Rectangle((rect['x'], rect['y']), rect['w'], rect['h'], linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(patch)
            ax.text(rect['x'], rect['y'] - 6, label, fontsize=10, color='white', backgroundcolor='black')
    plt.tight_layout()
    return fig

In [7]:
def build_vision_context(analysis_json):
    caption = ""
    if 'description' in analysis_json:
        captions = analysis_json['description'].get('captions', [])
        if captions:
            caption = captions[0]['text']

    tags = []
    if 'tags' in analysis_json:
        tags = [t['name'] for t in analysis_json['tags']]

    vision_context = f"Caption: {caption}\nTags: {', '.join(tags)}"
    return vision_context, caption, tags

In [8]:
# import os
# import google.generativeai as genai

# genai.configure(api_key=GENAI_API_KEY)

# print("Gemini configured ✅")

In [9]:
# Note: this uses the google.genai python client pattern you used in your original cells.
# If the package is not available in the environment, the user should pip install it or replace
# with a HuggingFace / OpenAI client of their choice.

try:
    import os
    import google.generativeai as genai
    genai.configure(api_key=GENAI_API_KEY)
    genai_client = genai.Client(api_key=GENAI_API_KEY)
except Exception as e:
    genai_client = None
    print("Warning: google.genai client not available in this environment.\nMake sure to pip install google-genai or adapt to another client.")


def generate_creative_from_context(vision_context, model_name="gemini-2.5-flash"):
    prompt = f"""
Write a short creative paragraph inspired by this image.
Use the following vision context:

{vision_context}

Make the paragraph imaginative, descriptive, and coherent.
"""

    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text


def transform_style_with_genai(original_paragraph, target_style="Arabic poetic style", model_name="gemini-2.5-flash"):
    style_prompt = f"""
Rewrite the following paragraph in {target_style}:

{original_paragraph}

Make sure the new version reflects the chosen style, changing tone, mood, vocabulary, and structure accordingly.
"""

    model = genai.GenerativeModel(model_name)
    response = model.generate_content(style_prompt)
    return response.text


Make sure to pip install google-genai or adapt to another client.


In [10]:
print(generate_creative_from_context("A cat sitting on a couch"))


The plush velvet couch, a sprawling kingdom of comfort, became a throne for the feline monarch. Bathed in a shaft of warm afternoon sun, his fur shimmered like spun gold, each relaxed muscle a testament to the profound art of doing absolutely nothing. Through half-lidded eyes, he surveyed his domain with the silent, ancient wisdom of cats, dreaming perhaps of grander hunts or simply perfecting the subtle purr that vibrated through the very cushions, a quiet symphony of contentment.


In [11]:
def translate_text_azure(text, to_language='ar'):
    # translator endpoint expects a POST to the translate?api-version=3.0&to={lang}
    url = AZURE_TRANSLATOR_ENDPOINT + f"/translate?api-version=3.0&to={to_language}"
    headers = {
        "Ocp-Apim-Subscription-Key": AZURE_TRANSLATOR_KEY,
        "Ocp-Apim-Subscription-Region": "francecentral", 
        "Content-Type": "application/json"
    }
    body = [{ 'text': text }]
    resp = requests.post(url, headers=headers, json=body)
    resp.raise_for_status()
    data = resp.json()
    # Azure returns a list with translations
    try:
        return data[0]['translations'][0]['text']
    except Exception:
        return json.dumps(data, ensure_ascii=False)

In [12]:
# Helper to accept either URL or uploaded PIL image and produce bytes

def pil_to_bytes(img_pil, fmt='JPEG'):
    bio = BytesIO()
    img_pil.convert('RGB').save(bio, format=fmt)
    return bio.getvalue()

In [13]:
def create_gradio_app():
    import gradio as gr

    def pipeline_run(image_input, image_url_input, style_choice, translate_to):
        """Main pipeline that accepts either an uploaded image (PIL) or an image URL.
        Returns: pil image (for display), vision_context (caption+tags), creative text, styled text, translated text
        """
        # Decide source
        image_pil = None
        analysis = {}
        if image_input is not None:
            # image_input is PIL.Image
            image_pil = image_input
            img_bytes = pil_to_bytes(image_pil)
            analysis = analyze_image_azure(image_bytes=img_bytes)
        elif image_url_input:
            image_pil = Image.open(BytesIO(requests.get(image_url_input).content))
            analysis = analyze_image_azure(image_url=image_url_input)
        else:
            return None, "", "", "", ""

        vision_context, caption, tags = build_vision_context(analysis)

        # creative generation
        creative = generate_creative_from_context(vision_context)

        # styled
        styled = transform_style_with_genai(creative, target_style=style_choice)

        # translate
        translated = translate_text_azure(styled, to_language=translate_to)

        return image_pil, vision_context, creative, styled, translated

    # Build Gradio UI
    with gr.Blocks() as demo:
        gr.Markdown("# AI Vision Pipeline — Image → Creative → Styled → Translated")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("**Input**")
                image_upload = gr.Image(label="Upload or drag & drop an image", type="pil")
                
                image_url = gr.Textbox(label="Or paste an image URL (leave empty if uploading)", placeholder="https://example.com/photo.jpg")
                style_dropdown = gr.Dropdown(label="Choose target style for rewriting",
                                            choices=["Arabic poetic style","Marketing advertising style","Child-friendly storytelling style","Journalistic tone","Casual conversational"],
                                            value="Arabic poetic style")
                translate_dropdown = gr.Dropdown(label="Translate styled text to:", choices=["en","ar","fr","es","de"], value="ar")
                run_btn = gr.Button("Analyze & Generate")

            with gr.Column(scale=1):
                gr.Markdown("**Outputs**")
                out_image = gr.Image(label="Image", type='pil')
                out_vision = gr.Textbox(label="Caption + Tags", lines=3)
                out_creative = gr.Textbox(label="Creative paragraph", lines=6)
                out_styled = gr.Textbox(label="Styled paragraph", lines=6)
                out_translated = gr.Textbox(label="Translated styled text", lines=6)

        # Wire up
        run_btn.click(fn=pipeline_run, inputs=[image_upload, image_url, style_dropdown, translate_dropdown],
                      outputs=[out_image, out_vision, out_creative, out_styled, out_translated])

    return demo


In [14]:
if __name__ == '__main__':
    app = create_gradio_app()
    # In Azure ML you may want to set share=False or True depending on setup
    app.launch(share=False, inbrowser=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [15]:
app.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://4b39356326bace7643.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
